# ExpAn: Experiment Analysis

ExpAn is a Python library for the statistical analysis of randomised controlled trials (A/B tests). 

The functions are standalone and can be imported and used from within other projects, and from the command line.

The library is Open Source, published under the MIT license here:

[github.com/zalando/expan](https://github.com/zalando/expan)

## Assumptions used in analysis 

1. Sample-size estimation:
  * Treatment does not affect variance
  * Variance in treatment and control is identical
  * Mean of delta is normally distributed
2. Welch t-test:
  * Mean of means is t-distributed (or normally distributed) 
3. In general:
  * Sample represents underlying population
  * Entities are independent

## Main user stories

As a Data Scientist I want to perform all the basic analysis routines that are typical of a the analysis of an A/B Test (a.k.a. Between-Subject Randomised Control Trial) while retaining access to the raw data so I can perform very also custom analyses in order to answer the questions of stakeholders with little effort.

As an analyst from a different department, I want to be able to bring my own data, and easily be able to use this library to perform analysis.

# Installation

To install the library:

    $ pip install expan

For more information, start with the [README.rst](https://github.com/zalando/expan/blob/master/README.rst)

# ExpAn Architecture


## `data.loaders` seperate details of data from the library

Loaders read the raw data (e.g. **`csv_fetcher.py`**) and construct an `Experiment` object.

## `core.experiment` provides the analysis functionality

**`Experiment`** class provides methods for the analysis of experiment data. Currently we support only the **`deltaKPI`** computation.

## `core.statistics` contains underlying statistical functions

**`Statistics`** class provides methods for statistical computations such as: **`delta`** - computes the difference of means between the samples (x-y) with the confidence intervals, **`bootstrap`** - confidence intervals boostrapping, **`chi-square`** - chi-square homogeneity test on categorical data. 

The class is used by higher-level `experiment` module, and can be used directly from CLI, by passing in `Array`s.

## `core.binning` implements categorical and numerical binnings

The class keeps binning separate from the data.

**`Binning`** class has two subclasses `NumericalBinning` and `CategoricalBinning`. `NumericalBinning` groups data into numerical bins defined by numerical intervals. `CategoricalBinning` bins data into categories. This methods provides binning implementations which can be applied to unseed data as well.

## `core.utils` contains supplied utility methods shared by other classes

Currently it supports methods for generating random data for performing an experiment.

## `core.version` constructs versioning of the package

# Data loaders

Data loaders can be written as needed to handle different formats (CSV, Parquet, HDF5, etc) and different internal structures, so long as they return an `ExperimentData` object.

Currently, only a simply CSV loader (`data.csv_fetcher`) has been implemented.

We'll bypass this and work with synthesized data for now:

In [2]:
import sys, os
import numpy as np
import pandas as pd

from expan.core.util import generate_random_data
from os.path import dirname, join, realpath
sys.path.insert(0, join(os.getcwd(), 'tests'))

np.random.seed(0)
data,metadata = generate_random_data()

In [3]:
data.head()

,entity,variant,normal_same,normal_shifted,feature,normal_shifted_by_feature,treatment_start_time,normal_unequal_variance
0,0,A,-1.487862,-0.616148,non,-1.088533,7,0.003991
1,1,B,-1.125186,1.783682,has,1.167307,3,-3.565511
2,2,B,0.388819,1.007539,non,-1.055948,1,6.704536
3,3,A,-1.173873,-0.889252,non,-0.152459,4,1.209668
4,4,A,1.112634,0.434377,has,0.175988,4,0.148207


In [3]:
metadata

{'experiment': 'random_data_generation',
 'primary_KPI': 'normal_shifted',
 'source': 'simulated'}

# Constructing `Experiment` 

The `Experiment` class has the following parameters to construct an experiment:

| Parameter | Description |
|---|---|
| **control_variant_name** | Indicates which of the variants is to be considered as a baseline (a.k.a. control). |
| **data** | A data you want to run experiment for. An example of the data structure see above. |
| **metadata** | Specifies an experiment name as the mandatory and data source as the optional fields. |
| **report_kpi_names** | A list of strings specifying desired kpis to analyse (empty list by default). |
| **derived_kpis** | A dictionary structured **{'name': ' `<`name_of_the_kpi`>`, 'formula': `<`formula_to_compute_kpi`>`}** (empty list by default) or a list of such dictionaries if more than 1 derived_kpi is wanted. **`<`name_of_the_kpi`>`**: name of the kpi. **`<`formula_to_compute_kpi`>`**: formula to calculate the desired kpi.|
    
**NOTE 1**. You should be careful specifying the correct structure to the derived_kpis dictionary including keys **'name'** and **'formula'**. Otherwise, construction of `Experiment` object will raise an exception.

**NOTE 2**. Specify the derived kpi name in the **report_kpi_names** if you want to see the results for it too.

**NOTE 3**. **data** must contain a column **entity**, a column **variant** and one column each for the kpis you defined.

**NOTE 4**. Fields in **metadata** see below.

```metadata``` should contain the following fields. Optional fields are wrapped in brackets.

| Field | Description |
|---|---|
|**`experiment`**| Name of the experiment, as known to stakeholders. It can be anything meaningful to you. |
|**`[sources]`**| Names of the data sources used in the preparation of this data.|
|**`[experiment_id]`**| This uniquely identifies the experiment. Could be a concatenation of the experiment name and the experiment start timestamp. |
|**`[retrieval_time]`**| Time that data was fetched from original sources... perhaps this should be a list with entry per source? |
|**`[primary_KPI]`**| Overall Evaluation Criteria. |

In [4]:
from expan.core.experiment import Experiment

exp = Experiment(control_variant_name='A', 
                 data=data, 
                 metadata=metadata, 
                 report_kpi_names=['derived_kpi_one'],
                 derived_kpis=[{'name':'derived_kpi_one','formula':'normal_same/normal_shifted'}])

In [5]:
print(exp)

Experiment "random_data_generation" with 1 derived kpis, 1 report kpis, 10000 entities and 2 variants: *A*, B


The wrong input structure (e.g. missing derived_kpis dictionary keys or incorrect kpi keys) will raise an exception.

In [6]:
exp = expan.experiment.Experiment(control_variant_name='A',
                                  data=data, 
                                  metadata=metadata,
                                  report_kpi_names=['normal_shifted', 'normal_same'],
                                  derived_kpis=[{'name':'derived_kpi_1'}])

KeyError: 'Dictionary should have key "formula"'

In our data we have two variants and one them is a baseline or control:

In [7]:
print('Variants: {}'.format(exp.variant_names))
print('Control or baseline variant: {}'.format(exp.control_variant_name))

Variants: {'A', 'B'}
Control or baseline variant: A


# Now we can start analysing!

### Let's start with a single delta analysis:

In [8]:
import warnings
import json

warnings.simplefilter('once', UserWarning)

In [9]:
res_delta = exp.delta()
print(json.dumps(res_delta, indent=2))

{
  "warnings": [
    "kpi: derived_kpi_one, variant: B: Sample variances differ too much to assume that population variances are equal."
  ],
  "errors": [],
  "expan_version": "0.6.2",
  "control_variant": "A",
  "kpis": [
    {
      "name": "derived_kpi_one",
      "variants": [
        {
          "name": "A",
          "delta_statistics": {
            "delta": 0.0,
            "confidence_interval": [
              {
                "percentile": 2.5,
                "value": -6.445256794169719
              },
              {
                "percentile": 97.5,
                "value": 6.445256794169717
              }
            ],
            "treatment_sample_size": 6108,
            "control_sample_size": 6108,
            "treatment_mean": -4.572524000045541,
            "control_mean": -4.572524000045541,
            "statistical_power": 0.050000000000000044
          }
        },
        {
          "name": "B",
          "delta_statistics": {
            "delta": 4.564

Currently **`delta`** supports 4 methods to compute `delta`: `fixed_horizon` (default), `group_sequential`, `bayes_factor` and `bayes_precision`. All methods requires different additional parameters.

**`fixed_horizon`** is a default method which has default settings/parameters:

| Parameter | Description |
|---|---|
|`assume_normal=True`| Specifies whether normal distribution assumptions can be made.|
|`percentiles=[2.5, 97.5]`| A list of percentile values for confidence bounds.|
|`min_observations=20`| Minimum number of observations needed.|
|`nruns=10000`| Only used if assume normal is false.|
|`relative=False`| If relative==True, then the values will be returned as distances below and above the mean, respectively, rather than the absolute values.|

**`group_sequential`**:

| Parameter | Description |
|---|---|
|`spending_function='obrien_fleming`| Currently we support only Obrient-Fleming alpha spending function for the frequentist early stopping decision.|
|`estimated_sample_size=None`| Sample size to be achieved towards the end of experiment.|
|`alpha=0.05`| Type-I error rate.|
|`cap=8`| Upper bound of the adapted z-score.|


**`bayes_factor`**:

| Parameter | Description |
|---|---|
|`distribution='normal'`| The name of the KPI distribution model, which assumes a Stan model file with the same name exists.`*`
|`num_iters=25000`| Number of iterations of bayes sampling.

**`bayes_precision`**:

| Parameter | Description |
|---|---|
|`distribution='normal'`| The name of the KPI distribution model, which assumes a Stan model file with the same name exists.`*`
|`posterior_width=0.08`| The stopping criterion, threshold of the posterior width.
|`num_iters=25000`| Number of iterations of bayes sampling.

`*` currently we support **`normal`** and **`poisson`** models.

If you would like to change any of the default values, just pass them as parameters to delta:

In [10]:
delta_freq = exp.delta(method='fixed_horizon', assume_normal=True, percentiles=[2.5, 99.5])

In [11]:
delta_g_s = exp.delta(method='group_sequential', estimated_sample_size=1000)

In [12]:
delta_bayes_factor = exp.delta(method='bayes_factor', distribution='normal')

In [13]:
print(json.dumps(delta_bayes_factor, indent=2))

{
  "warnings": [
    "kpi: derived_kpi_one, variant: B: Sample variances differ too much to assume that population variances are equal."
  ],
  "errors": [],
  "expan_version": "0.6.2",
  "control_variant": "A",
  "kpis": [
    {
      "name": "derived_kpi_one",
      "variants": [
        {
          "name": "A",
          "delta_statistics": {
            "stop": true,
            "delta": 0.0,
            "confidence_interval": [
              {
                "percentile": 2.5,
                "value": -8.396765530428699
              },
              {
                "percentile": 97.5,
                "value": 3.5794735964894677
              }
            ],
            "treatment_sample_size": 6108,
            "control_sample_size": 6108,
            "treatment_mean": -4.572524000045541,
            "control_mean": -4.572524000045541,
            "number_of_iterations": 25000,
            "statistical_power": 0.050000000000000044
          }
        },
        {
          "

# Interpreting the output

| Metric | Description |
|---|---|
|**`treatment_mean`**| the mean of the treatment group |
|**`control_mean`**| the mean of the control group |
|**`control_sample_size`**| the sample size for the control group |
|**`treatment_sample_size`**| the sample size for the treatment group |
|**`delta`**| the difference between the treatment_mean and control_mean |
|**`confidence_interval`**| the confidence interval: **`percentile`** - lower percentile and upper percentile; **`value`** - value for each percentile |
|**`number_of_iterations`**| number of iterations used for bayes sampling for **`bayes_factor`** and **`bayes_precision`** methods|
|**`stop`**| when **`True`** means the early stopping was done|
|**`statistical_power`**| statistical power --- that is, the probability of a test to detect an effect, if the effect actually exists|

# Using Bootstrapping:

We implement boostrapping for data which is not normally distributed.

We switch the flag 'assume_normal' to False for the `delta` function:

In [14]:
res_delta = exp.delta(assume_normal=False)
print(json.dumps(res_delta, indent=2))

{
  "warnings": [
    "kpi: derived_kpi_one, variant: B: Sample variances differ too much to assume that population variances are equal."
  ],
  "errors": [],
  "expan_version": "0.6.2",
  "control_variant": "A",
  "kpis": [
    {
      "name": "derived_kpi_one",
      "variants": [
        {
          "name": "A",
          "delta_statistics": {
            "delta": 0.0,
            "confidence_interval": [
              {
                "percentile": 2.5,
                "value": -6.451719231491042
              },
              {
                "percentile": 97.5,
                "value": 6.412118814781721
              }
            ],
            "treatment_sample_size": 6108,
            "control_sample_size": 6108,
            "treatment_mean": -4.572524000045541,
            "control_mean": -4.572524000045541,
            "statistical_power": 0.050000000000000044
          }
        },
        {
          "name": "B",
          "delta_statistics": {
            "delta": 4.564

You may not notice here: bootstrapping takes considerably longer time than assuming the normality before running experiment. If we do not have an explicit reason to use it, it is almost always better to leave it off.

# Binning

You can use the Binning module to group data into subsets, i.e., assign each data into a corresponding `Bin` object. We will explain respectively in the next few sections.

## Create bin object directly
If you already know the set of bins you want to put data into. You can initialize a `bin` object directly.

The first argument is the id of the bin. This might not be useful for your application but serves as a technical identifier.
The second argument is the type of the bin. This can either be "numerical" or "categorical". Depending on the type, you should pass coresponding representation object as the third argument.

We will explain the representation in next section.

## Numerical representation

In [2]:
from expan.core.binning import *

# numerical bin
# create a numerical bin from value 0 (inclusive) to 10 (exclusive).
Bin("numerical", 0, 10, True, False)

ExpAn core init: v0.6.2



bin: [0, 10)

## Categorical representation

In [16]:
# categorical bin
# create a categorical bin which contains categories of "a" and "b".
Bin("categorical", ["a", "b"])


bin: ['a', 'b']

## Create bin object automatically

Given a number of bins, you can also create a list of bins from data by using the method ```create_bins(data, n_bins)```. 

It will create ```n_bins``` Bin ojbects, by which to separate the ```data``` as equally as possible. This method will also automatically detects numerical or categorical data, and creates corresponding bin representations.

In [17]:
data1 = exp.data[exp.data.variant == 'A']
data2 = exp.data[exp.data.variant == 'B']

In [18]:
n_bins = 10
create_bins(data1.normal_same, n_bins)

[
bin: [-3.83665554846, -1.25906491145), 
bin: [-1.25906491145, -0.804751813719), 
bin: [-0.804751813719, -0.489466995342), 
bin: [-0.489466995342, -0.226662203724), 
bin: [-0.226662203724, 0.0239463824493), 
bin: [0.0239463824493, 0.276994331119), 
bin: [0.276994331119, 0.551060124216), 
bin: [0.551060124216, 0.868798338306), 
bin: [0.868798338306, 1.30062540106), 
bin: [1.30062540106, 4.47908425103]]


## Assign data to bins
We can use the method ```apply(data):``` of the ```Bin``` object to assign data to one of the given bins.

This method will return a subset of input data which belongs to this bin.
It will return ```None``` if there is no data matched for this bin.

In [21]:
a_bin = Bin("numerical", 0, 10, True, False)
print("applying bin to data in variant A:")
a_bin.apply(data1.normal_same)

applying bin to data in variant A:


4       1.112634
6       0.085595
10      0.335054
13      0.542203
15      0.002232
19      0.467690
21      1.171102
23      1.289203
27      0.141980
28      0.313723
31      1.345935
37      2.418778
41      0.288028
44      0.411566
46      1.120967
47      0.805575
48      0.975823
49      0.008858
54      1.352039
57      2.159121
58      0.091315
61      1.637082
63      0.735269
66      1.030250
71      0.644690
77      0.723038
78      0.085513
83      1.889279
84      0.238171
89      0.580568
          ...   
9873    0.030269
9875    0.863606
9876    0.524865
9880    0.008274
9891    0.395712
9900    1.168769
9901    0.055230
9903    0.192369
9908    0.010693
9909    0.354407
9910    0.853060
9914    0.492523
9918    0.502002
9924    1.096724
9925    0.688108
9934    0.367047
9935    0.279812
9936    0.445043
9945    0.876760
9948    0.261577
9954    1.601119
9955    1.797017
9959    0.542985
9969    0.206816
9973    1.589447
9980    0.130357
9982    0.377618
9985    0.1936

In [22]:
print("applying bin to data in variant B:")
a_bin.apply(data2.normal_same)

applying bin to data in variant B:


2       0.388819
8       0.772848
9       0.783160
11      0.564789
20      1.310606
25      0.600733
30      0.608267
33      1.360168
35      0.849585
38      1.495458
43      0.444854
51      0.977872
55      2.099408
69      1.132805
70      1.597397
74      0.079915
75      0.320930
86      0.220631
88      0.324758
92      1.638961
104     1.277857
107     1.498012
115     1.344854
118     2.120994
127     0.059905
139     2.254038
156     0.079048
161     0.150602
165     0.090310
170     0.947512
          ...   
9862    0.725924
9863    1.492610
9864    0.908889
9883    1.138699
9885    0.167043
9886    0.285282
9887    0.322020
9894    2.127297
9897    1.896604
9911    1.127925
9913    0.499415
9915    0.327819
9927    0.729370
9928    0.887623
9937    0.278923
9938    0.729843
9940    0.201785
9943    1.338250
9957    0.544323
9958    0.858663
9971    0.290580
9972    1.081581
9977    0.460328
9981    0.084888
9983    0.443676
9984    0.338594
9989    1.544333
9993    0.6726

# Subgroup Analysis

Subgroup analysis in ExaAn will select subgroup (which is a segment of data) based on the input argument, and then perform a regular delta analysis per subgroup as described before. 
That is to say, we don't compare between subgroups, but compare treatment with control within each subgroup.

The input argument is a ```dict```, which maps feature name (key) to a list of ```Bin``` objects (value). This ```dict``` defines how and on which feature to perform the subgroup split. 
The returned value of subgroup analysis will be the result of regular delta analysis per subgroup.

An example is provided below.

In [8]:
dimension_to_bins = {"treatment_start_time": [
    Bin("numerical", 0, 5, True, False), 
    Bin("numerical", 5, 10, True, False)]
}
exp.sga(dimension_to_bins)

[{'dimension': 'treatment_start_time',
  'result': {'control_variant': 'A',
   'errors': [],
   'expan_version': '0.6.2',
   'kpis': [{'name': 'derived_kpi_one',
     'variants': [{'delta_statistics': {'confidence_interval': [{'percentile': 2.5,
          'value': -1.5569210692070499},
         {'percentile': 97.5, 'value': 2.1978673629800363}],
        'control_mean': -0.32639393302612346,
        'control_sample_size': 3076,
        'delta': 0.3204731468864935,
        'statistical_power': 0.095063282824786377,
        'treatment_mean': -0.005920786139629961,
        'treatment_sample_size': 1930},
       'name': 'B'},
      {'delta_statistics': {'confidence_interval': [{'percentile': 2.5,
          'value': -2.1025221680926345},
         {'percentile': 97.5, 'value': 2.102522168092634}],
        'control_mean': -0.32639393302612346,
        'control_sample_size': 3076,
        'delta': 0.0,
        'statistical_power': 0.050000000000000044,
        'treatment_mean': -0.3263939330261

# Statistics

Here the underlying statistical functions are implemented. These are used by the higher-level `experiment` module, and can indeed be used directly by passing in NumPy `Array`s.

The more interesting functions are:

###  `bootstrap`

Bootstraps the Confidence Intervals for a particular function comparing two samples. NaNs are ignored (discarded before calculation).

This function, as well as others such as `normal_sample_difference`, and `delta`, take as input a list of percentiles, and return the values corresponding to those percentiles. This implementation is very general, allowing us to use the same functions for one-sided as well as two-sided tests, as well as more exactly recreating an output distribution (e.g. if we want to graphically depict more than 95% confidence intervals).

### `delta`

Uses either bootstrap or standard normal assumptions to compute the difference between two arrays.

# That's it! Try it out for yourself:


[github.com/zalando/expan](https://github.com/zalando/expan)